<a href="https://colab.research.google.com/github/Sakitha/Unstractured_PDF_DATA/blob/master/pharser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print("Version: ", tf.__version__)
import tensorflow_addons as tf_ad
import numpy as np

Version:  2.3.0


In [ ]:
'''
lets say a sample cv has the following format;

X K 
  V
L

if we choose following words to be our corner words,  
corner_words = X,V,J,L

features ->

 ------X-----    ------V-----    ------J-----  ------L-----
[[x_X, y_X, f_X, x_V, x_V, f_V, x_J, y_J, f_J ,x_L, y_L, f_L ],
[x_X, y_X, f_X, x_V, x_V, f_V, x_J, y_J, f_J ,x_L, y_L, f_L ],
[x_X, y_X, f_X, x_V, x_V, f_V, x_J, y_J, f_J ,x_L, y_L, f_L ],
[x_X, y_X, f_X, x_V, x_V, f_V, x_J, y_J, f_J ,x_L, y_L, f_L ]]
'''

'\nlets say a sample cv has the following format;\n\nX K \n  V\nL\n\nif we choose following words to be our corner words,  \ncorner_words = X,V,J,L\n\nfeatures ->\n\n ------X-----    ------V-----    ------J-----  ------L-----\n[[x_X, y_X, f_X, x_V, x_V, f_V, x_J, y_J, f_J ,x_L, y_L, f_L ],\n[x_X, y_X, f_X, x_V, x_V, f_V, x_J, y_J, f_J ,x_L, y_L, f_L ],\n[x_X, y_X, f_X, x_V, x_V, f_V, x_J, y_J, f_J ,x_L, y_L, f_L ],\n[x_X, y_X, f_X, x_V, x_V, f_V, x_J, y_J, f_J ,x_L, y_L, f_L ]]\n'

In [ ]:
class NerModel(tf.keras.Model):
    def __init__(self, hidden_num, label_size):
        super(NerModel, self).__init__()

        self.num_hidden = hidden_num
        self.label_size = label_size

        self.dense1 = tf.keras.layers.Dense(hidden_num, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(100, activation=tf.nn.relu)
        self.dense3 = tf.keras.layers.Dense(label_size, activation=tf.nn.softmax)
        self.transition_params = tf.Variable(tf.random.uniform(shape=(label_size, label_size)))

    def call(self, inputs, labels=None, training=None):
        ta = tf.TensorArray(tf.float32, size=0, dynamic_size=True, infer_shape=False)
        word_count = inputs.shape[1]

        for time_step in range(word_count):
            input = inputs[:, time_step, :]
            x1 = self.dense1(input)
            x2 = self.dense2(x1)
            ffd_output = self.dense3(x2)
            ta = ta.write(time_step, ffd_output)

        logits = tf.transpose(ta.stack(), [1, 0, 2])
        if labels is not None:
            label_sequences = tf.convert_to_tensor(labels, dtype=tf.int32)
            log_likelihood, self.transition_params = tf_ad.text.crf_log_likelihood(logits, label_sequences,
                                                                               [word_count],
                                                                               transition_params=self.transition_params)
            return logits, [word_count], log_likelihood
        else:
            return logits,[word_count]

In [ ]:
def train_one_step(text_batch, labels_batch):
    with tf.GradientTape() as tape:
        logits, text_lens, log_likelihood = model(text_batch, labels_batch, training=True)
        loss = - tf.reduce_mean(log_likelihood)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, logits, text_lens

In [ ]:
def get_acc_one_step(logits, text_lens, labels_batch):
    paths = []
    accuracy = 0
    y_real = []
    for logit, text_len, labels in zip(logits, text_lens, labels_batch):
        viterbi_path, _ = tf_ad.text.viterbi_decode(logit[:text_len], model.transition_params)
        paths.append(viterbi_path)
        if type(labels) is np.ndarray:
            y_real.append(list(labels[:text_len]))
        else:
            y_real.append(list(labels.numpy()[:text_len]))

        correct_prediction = tf.equal(
            tf.convert_to_tensor(tf.keras.preprocessing.sequence.pad_sequences([viterbi_path], padding='post'),
                                 dtype=tf.int32),
            tf.convert_to_tensor(tf.keras.preprocessing.sequence.pad_sequences([labels[:text_len]], padding='post'),
                                 dtype=tf.int32)
        )

        accuracy = accuracy + tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    accuracy = accuracy / len(paths)
    return accuracy

In [ ]:
inputs = np.asarray([[[.1,.2,.3,.4],[.1,.2,.3,.4],[.1,.2,.3,.4]]], dtype=np.float32)
targets   = np.asarray([[1,2,2]])

model = NerModel(hidden_num=512, label_size=3)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

for epoch in range(10):
    loss, logits, text_lens = train_one_step(inputs, targets)
    accuracy = get_acc_one_step(logits, text_lens, targets)
    print('epoch %d, loss %.4f , accuracy %.4f' % (epoch, loss, accuracy))


epoch 0, loss 3.7015 , accuracy 0.0000
epoch 1, loss 3.4594 , accuracy 0.0000
epoch 2, loss 3.1747 , accuracy 0.3333
epoch 3, loss 2.9630 , accuracy 0.3333
epoch 4, loss 2.8838 , accuracy 0.3333
epoch 5, loss 2.8512 , accuracy 0.3333
epoch 6, loss 2.8259 , accuracy 0.3333
epoch 7, loss 2.8016 , accuracy 0.3333
epoch 8, loss 2.7776 , accuracy 0.3333
epoch 9, loss 2.7538 , accuracy 0.3333
